In [1]:
import pandas as pd
import psycopg2
import matplotlib
import numpy as np
import os

conn = psycopg2.connect("dbname=hmdamaster user=roellk") #connect and return connection
cur = conn.cursor()

#property type appears in 2004
#FIXME reload and clean 2004

#set names of PostGRES HMDA data tables
#App and Orig tables are aggregates from the LAR tables

hmda_app_tables = ['county_apps_2000','county_apps_2001','county_apps_2002','county_apps_2003','county_apps_2004','county_apps_2005',
                  'county_apps_2006','county_apps_2007','county_apps_2008','county_apps_2009','county_apps_2010','county_apps_2011',
                  'county_apps_2012','county_apps_2013','county_apps_2014']

hmda_orig_tables = ['county_orig_2000','county_orig_2001','county_orig_2002','county_orig_2003','county_orig_2004','county_orig_2005',
                  'county_orig_2006','county_orig_2007','county_orig_2008','county_orig_2009','county_orig_2010','county_orig_2011',
                  'county_orig_2012','county_orig_2013','county_orig_2014']

hmda_lar_tables = ['hmdalar2000','hmdalar2001','hmdalar2002','hmdalar2003','hmdalar2004','hmdalar2005','hmdalar2006','hmdalar2007',
                  'hmdalar2008','hmdalar2009','hmdalar2010','hmdalar2011','hmdalar2012','hmdalar2013','hmdalar2014']


#count of loans
#average and median dollar value of loans
#loan value distribution measure
#dollar volume of loan amount
#demographic count, value, origination rates
#income change
#compare income change to value change in loan amount

In [2]:
#load tract to CBSA file to get valid counties
cbsa_df = pd.read_csv('tract_to_CBSA_2010.csv', sep='|')
cbsa_df.county = cbsa_df.county.map(lambda x: str(x).zfill(5)) #left pad with 0's to make valid FIPS codes
fips_list = set(cbsa_df.county.ravel()) #remove duplicates


In [3]:
#selects application and origination information for single family homes for a single county
#loads to data frame and writes to csv in state/county directory structure
#writes a CSV to the base directory with all counties and all states
#this process takes ~15 minutes to run

#select data for 2000-2014
#FIXME: create a file containing all counties for all years

for fips in list(set(fips_list))[:100]:
    first = True
    state = fips[:2]
    county = fips[2:]
    path = "data/"+ state+"/"+fips
    for app_table in hmda_app_tables: #loop over county aggregate year tables
        orig_table = table[:7]+"orig"+app_table[len(table)-5:] #change name of table to hmda_orig_YYYY
        SQL = """
            SELECT * 
            FROM {app_table} AS app
            JOIN {orig_table} AS orig
            ON app.fips = orig.fips
            WHERE app.fips = cast({fips} AS VARCHAR(5))
            
            """
        SQL = SQL.format(app_table=app_table, orig_table=orig_table, fips=fips)
        df = pd.read_sql_query(SQL, conn) #load query results to dataframe
        
        if first:
            out_file = df #establish outfile with first year
            first = False
        else:
            out_file = pd.concat([out_file, df]) #append a year to a county dataframe
            #print df.head()
            
    if not os.path.exists(path): #check to see if path for a county exists
            os.makedirs(path) #create path if it is not present
    out_file.to_csv(path+"/data.csv", index=None) #write dataframe to CSV for one county in state/county/file directory structure
        




NameError: name 'table' is not defined

In [ ]:
#create county files and put them in directory tree

bad_counties = [] #keep a list of counties that were not valid FIPS
good_counties = [] #make a list of valid Census counties

for fips in fips_list: #FIXME change to relevant list
    

    path = "data/"+str(fips)[:2] + "/" + str(fips) #set CSV path to data/state/county format
#    print path
    if os.path.exists(path): #not all valid FIPS codes will have mortgage activity
        out_df = pd.read_csv(path + "/data.csv")
        out_df.head()
        #create income multiple for single year
        out_df['income_multiple'] = (out_df.loan_average_app / 0.80) / out_df.income_average_app
        #create deltas for pattern building
        out_df['loan_avg_app_delta'] = out_df.loan_average_app.pct_change()*100
        out_df['income_average_app_delta'] = out_df.income_average_app.pct_change() *100
        out_df['count_app_delta'] = out_df.count_app.pct_change() *100
        out_df['value_app_delta'] = out_df.value_app.pct_change()
        out_df['income_mult_delta'] = out_df.income_multiple.pct_change()*100

        #write CSV
        out_df.to_csv(path + '/data.csv', index=None)
        good_counties.append(str(county))
    else:
        #print "bad county: ", county, len(county)
        bad_counties.append(county)